In [14]:
import psycopg2
import pandas as pd
import openpyxl
import csv
import os
import io

In [15]:
conn = psycopg2.connect(
    dbname="testdb",
    user="postgres",
    password="admin",
    host="localhost",
    port=5432
)

In [17]:
cursor = conn.cursor()
file_path = r"C:\Users\ansh\Desktop\tracks.xlsx"

In [21]:
cursor.execute("""
CREATE INDEX idx_tracks_isrc ON tracks(isrc);
               """)
conn.commit()

In [12]:
copy_sql = """
COPY musicalwork (
    UnclaimedMusicalWorkRightShareRecordId,
    ResourceRecordId,
    MusicalWorkRecordId,
    ISRC,
    DspResourceId,
    ResourceTitle,
    ResourceSubTitle,
    AlternativeResourceTitle,
    DisplayArtistName,
    DisplayArtistISNI,
    Duration,
    UnclaimedRightSharePercentage,
    PercentileForPrioritisation
)
FROM STDIN WITH (FORMAT csv, HEADER true, DELIMITER E'\t')
"""
with open(file_path, "r", encoding="utf-8") as file:
    cursor.copy_expert(copy_sql, file)

conn.commit()


In [82]:
with open(file_path, "r", encoding="utf-8") as f:
    for i in range(5):
        print(repr(f.readline()))


UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 15-16: invalid continuation byte

In [11]:
try:
    cursor.execute("""
    create table if not exists musicalwork (
        UnclaimedMusicalWorkRightShareRecordId bigint,
        ResourceRecordId TEXT,
        MusicalWorkRecordId TEXT,
        ISRC TEXT,
        DspResourceId TEXT,
        ResourceTitle text,
        ResourceSubTitle text,
        AlternativeResourceTitle text,
        DisplayArtistName text,
        DisplayArtistISNI text,
        Duration Numeric,
        UnclaimedRightSharePercentage float,
        PercentileForPrioritisation float
    );
    """)
    conn.commit()

except Exception as e:
    print("Error:", e)
    conn.rollback()


In [10]:
cursor.execute("Drop table musicalwork;")

In [78]:
cursor.close()
conn.close()

In [19]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS tracks (
    track_name TEXT,
    record TEXT,
    record_type TEXT,
    release_date TEXT,
    isrc TEXT,
    spotify_url TEXT,
    duration INT,
    popularity INT,
    explicit BOOLEAN
);
""")
conn.commit()

In [20]:
wb = openpyxl.load_workbook(file_path, data_only=True)
sheet = wb.active

csv_buffer = io.StringIO()
writer = csv.writer(csv_buffer)

for row in sheet.iter_rows(values_only=True):
    writer.writerow(row)

csv_buffer.seek(0)

copy_sql = f"COPY tracks FROM STDIN WITH CSV HEADER"
cursor.copy_expert(copy_sql, csv_buffer)
conn.commit()

csv_buffer.close()
wb.close()
sheet = None


In [18]:
cursor.execute("DROP TABLE Tracks;")
conn.commit()